In [7]:
%pip install -e ..

Obtaining file:///Users/sethvanderbijl/Coding%20Projects/DM_project_2
  Attempting uninstall: DM-project-2
    Found existing installation: DM-project-2 1.0
    Uninstalling DM-project-2-1.0:
      Successfully uninstalled DM-project-2-1.0
  Running setup.py develop for DM-project-2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from json import load
from data.load import load_data
import os
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, StackingClassifier, StackingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, HuberRegressor, LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import std
import time
import matplotlib.pyplot as plt
import copy
from helpers.ranking import to_ranking


/usr/local/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [44]:

# Load data
df = load_data(add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)

DF with these specific params is cached, returning cached version.


In [46]:
# Split into target and predictors
y = df['booking_bool']
X = df.drop(['booking_bool','click_bool', 'position', 'gross_bookings_usd', 'date_time'], axis=1)
X = X.fillna(X.mean())
del df

In [5]:
# Set up stakcing regressor
# rf = RandomForestRegressor()
# rf= StackingRegressor([('rf',RandomForestRegressor()), ('br',BayesianRidge()), ('ab',AdaBoostRegressor())], verbose=3, n_jobs=7)
rf=RandomForestRegressor(verbose=3)
rf.fit(X, y)

RandomForestRegressor()

In [14]:
import joblib
_ = joblib.dump(rf, 'model.pkl')

In [49]:
test = load_data(mode='our_test', add_day_parts=True, same_value_operations=[('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg')], fts_operations=[('prop_starrating', 'visitor_hist_starrating', 'diff')], add_seasons=True)


No cache for this specific request, start loading base df from disk.


100%|██████████| 6/6 [00:00<00:00, 40.31it/s]

generating dayparts
1 / 6 Generating datetimes for early_morning
2 / 6 Generating datetimes for morning
3 / 6 Generating datetimes for noon
4 / 6 Generating datetimes for eve
5 / 6 Generating datetimes for night
6 / 6 Generating datetimes for late_night



100%|██████████| 2/2 [00:00<00:00, 38.71it/s]


generating seasons
generating same id features
Feature engineering columns with operations
saving result of specific arguments to cache


100%|██████████| 500/500 [00:08<00:00, 61.13it/s]


In [57]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_test = test.drop(['date_time', 'gross_bookings_usd', 'booking_bool', 'click_bool', 'position'], axis=1)
X_test = X_test.fillna(X.mean()) #Mean of x or mean of x_test?

In [58]:
# Check if there is a difference in columns (there shouldn't be)
list(set(X_test.columns) - set(X.columns))

[]

In [63]:
from helpers.ndcg import ndcg_eval

preds = rf.predict(X_test)

ranking = to_ranking(X_test, preds)

score = ndcg_eval(ranking, test)

score


[0.16 0.24 0.07 ... 0.02 0.   0.05]
      srch_id  prop_id
14       2484    59097
1        2484     5355
7        2484    30762
4        2484    19438
31       2484   140724
...       ...      ...
2246   330758   123498
2226   330758    19251
2221   330758    13184
2239   330758    93651
2247   330758   134022

[2249 rows x 2 columns]


0.4436409815645396